<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!pip install gensim

In [2]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-03-04 19:15:16--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-03-04 19:15:16--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.2’

SO_vectors_200.bin? 100%[===================>]   1.35G  29.4MB/s    in 50s     

2024-03-04 19:16:07 (27.7 MB/s) - ‘SO_vectors_200.bin?download=1.2’ saved [1453905423/1453905423]



In [3]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [4]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [5]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [6]:
# method most_simmilar
wv_embeddings.most_similar(positive=['dog'])

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

In [7]:
wv_embeddings.most_similar(positive=['cat'], topn=5)

[('-vet', 0.7583354711532593),
 ('dog', 0.6852341890335083),
 ('/proc/version', 0.6815888285636902),
 ('feline', 0.6779966950416565),
 ('nyan', 0.6685197353363037)]

Ответ:

Слово "cat" не входит в топ 5 ближайших слов к "dog", но входит множественная форма слова "cats", расположившаяся на 4 месте.

А вот "dog" по близости к слову "cat" стоит на 2 месте.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [8]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
      return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [9]:
def question_to_vec(question, embeddings, tokenizer, lower = False, dim = 200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    if(lower == False):
      tokens = tokenizer.tokenize(question)
    else:
      tokens = tokenizer.tokenize(question.lower())

    vec = np.zeros(dim)
    num_words = 0

    for word in tokens:
      try:
        vec += embeddings[word]
        num_words += 1
      except:
        continue

    if num_words == 0:
      return vec

    return vec/num_words

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [10]:
print(round(question_to_vec("I love neural networks", wv_embeddings, tokenizer)[2], 2))

-1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ:

Hits@47 должно быть максимальным => N = 1 и номер дубликата < K => Max(Hits@47) = 1

DCG@1 должно быть минимальным => условие не должно выполняться, чтобы получился 0. Значит дубликат должен стоять, как минимум, на 2-ой позиции. Тогда DCG@1 = 0

Итого: Hits@47 - DCG@1 = 1 - 0 = 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [11]:
##DCG@10 = 1/log2 (1 + 9) * [9 <= 10]
import math

round(1/(math.log(1 + 9, 2)) * 1, 1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [12]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = 0
    for p_i in dup_ranks:
      if p_i > k:
        continue
      hits_value += 1

    hits_value /= len(dup_ranks)
    return hits_value

In [13]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = 0
    for p_i in dup_ranks:
      if p_i > k:
        continue
      dcg_value += 1/math.log(1+p_i, 2)

    dcg_value /= len(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [14]:
import pandas as pd

In [15]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

def find_duplicate(answer, candidates_ranking):
  for i, candidate in enumerate(candidates_ranking, 1):
    if candidate == answer:
      return i

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [find_duplicate(copy_answers[0], candidates_ranking[0])]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [17]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Считайте данные.

In [18]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split("\t"))
    return data

Нам понадобиться только файл validation.

In [19]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [20]:
len(validation_data)

3760

Размер нескольких первых строк

In [21]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [23]:
def rank_candidates(question, candidates, embeddings, tokenizer, lower = False, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_embedding = [question_to_vec(question, embeddings, tokenizer, lower, dim)]
    candidates_embeddings = [question_to_vec(candidate, embeddings, tokenizer, lower, dim) for candidate in candidates]

    cos_similarity = pd.Series(cosine_similarity(question_embedding, candidates_embeddings).squeeze()).sort_values(ascending=False)
    print(cos_similarity)

    return [(i, candidates[i]) for i in cos_similarity.index]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [24]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [25]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, False)
        print(ranks)
        print()

1    0.546291
0    0.478863
2    0.161885
dtype: float64
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

1    0.476891
0    0.456955
2    0.163290
dtype: float64
[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [26]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

SyntaxError: invalid syntax (<ipython-input-26-858aee104232>, line 5)

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ:

**Для эксперимента №2 была получена последовательность - 102**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [27]:
from tqdm.notebook import tqdm

In [28]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer, False)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
744    0.412025
         ...   
143   -0.196092
103   -0.206687
52    -0.208530
378   -0.216745
77    -0.230504
Length: 1000, dtype: float64
100    0.703191
828    0.686266
590    0.665650
692    0.665574
449    0.657387
         ...   
158   -0.115063
522   -0.142229
753   -0.153389
933   -0.154340
371   -0.186724
Length: 1000, dtype: float64
0      0.863638
479    0.743708
981    0.714146
692    0.679386
250    0.664649
         ...   
786   -0.126703
827   -0.128855
600   -0.136822
340   -0.151924
656   -0.152497
Length: 1000, dtype: float64
517    0.575234
928    0.523020
853    0.492112
843    0.474253
823    0.472424
         ...   
23    -0.097023
130   -0.108048
863   -0.128702
786   -0.144996
79    -0.193059
Length: 1000, dtype: float64
436    0.597205
257    0.593923
614    0.524159
283    0.521664
406    0.505479
         ...   
588   -0.140348
906   -0.150159
161   -0.150159
733   -0.155390
887   -0.213162


In [29]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.878
DCG@1000: 0.444 | Hits@1000: 1.000


In [30]:
## с использованием lower в токенизации
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer, True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
416    0.580346
         ...   
220   -0.219962
4     -0.220802
524   -0.238027
77    -0.241147
687   -0.251047
Length: 1000, dtype: float64
472    0.696516
744    0.677259
806    0.672096
100    0.655339
47     0.650872
         ...   
723   -0.101474
586   -0.108566
371   -0.125742
215   -0.130699
933   -0.158321
Length: 1000, dtype: float64
530    0.808768
301    0.799307
479    0.734897
396    0.729676
0      0.700331
         ...   
650   -0.136457
455   -0.138585
32    -0.142617
437   -0.142860
849   -0.212693
Length: 1000, dtype: float64
0      0.863598
517    0.795488
553    0.717196
386    0.710981
343    0.647362
         ...   
801   -0.124313
291   -0.127716
625   -0.129550
863   -0.134381
130   -0.141090
Length: 1000, dtype: float64
257    0.753189
427    0.693118
0      0.678752
316    0.670405
455    0.666706
         ...   
716   -0.151502
762   -0.168120
882   -0.170619
374   -0.175790
887   -0.210314


In [31]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.524 | Hits@  10: 0.650
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [32]:
train_data = read_corpus('./data/train.tsv')

In [33]:
tokenizer = MyTokenizer()

In [34]:
words = [(tokenizer.tokenize(' '.join(question))) for question in train_data]

In [35]:
len(words)

1000000

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

Берём значение скользящего окна = 5, беря в расчёт, что контекстуальные слова могут стоять поодаль от центрального слова.

In [36]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [37]:
## без lower
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
0      0.375802
         ...   
452   -0.233829
154   -0.247996
790   -0.251256
897   -0.266763
902   -0.299903
Length: 1000, dtype: float64
990    0.668315
445    0.667511
777    0.629019
47     0.628066
216    0.614612
         ...   
838   -0.188209
215   -0.225308
72    -0.240907
617   -0.293034
941   -0.293317
Length: 1000, dtype: float64
0      0.721514
530    0.686727
479    0.672206
4      0.644231
301    0.639691
         ...   
507   -0.149452
808   -0.160373
652   -0.164948
406   -0.166820
352   -0.206556
Length: 1000, dtype: float64
0      0.694987
476    0.642394
517    0.630747
73     0.625254
111    0.622588
         ...   
785   -0.175120
863   -0.178690
33    -0.181983
462   -0.193656
271   -0.199973
Length: 1000, dtype: float64
571    0.651908
751    0.602989
927    0.599991
627    0.591608
257    0.582671
         ...   
119   -0.169019
205   -0.175576
374   -0.195943
605   -0.209238
882   -0.231166


In [38]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.268 | Hits@   1: 0.268
DCG@   5: 0.333 | Hits@   5: 0.394
DCG@  10: 0.358 | Hits@  10: 0.471
DCG@ 100: 0.408 | Hits@ 100: 0.720
DCG@ 500: 0.435 | Hits@ 500: 0.926
DCG@1000: 0.443 | Hits@1000: 1.000


In [39]:
## с lower
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
201    0.452405
         ...   
652   -0.145693
859   -0.162152
184   -0.165375
897   -0.197814
902   -0.247319
Length: 1000, dtype: float64
990    0.693180
445    0.682826
100    0.673345
777    0.659542
915    0.659285
         ...   
215   -0.125714
838   -0.129306
598   -0.154737
617   -0.168750
293   -0.375733
Length: 1000, dtype: float64
530    0.762591
301    0.753593
479    0.741585
0      0.740122
981    0.688615
         ...   
148   -0.117692
808   -0.134334
876   -0.141818
126   -0.206341
652   -0.223281
Length: 1000, dtype: float64
0      0.726169
517    0.673543
476    0.654239
111    0.648565
553    0.628990
         ...   
785   -0.168815
923   -0.183060
462   -0.187477
271   -0.214185
297   -0.252733
Length: 1000, dtype: float64
571    0.696385
257    0.683731
927    0.673558
627    0.671309
751    0.669160
         ...   
586   -0.163136
540   -0.168167
605   -0.191891
882   -0.203415
374   -0.238017


In [40]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.267 | Hits@   1: 0.267
DCG@   5: 0.335 | Hits@   5: 0.397
DCG@  10: 0.357 | Hits@  10: 0.463
DCG@ 100: 0.410 | Hits@ 100: 0.720
DCG@ 500: 0.435 | Hits@ 500: 0.918
DCG@1000: 0.444 | Hits@1000: 1.000


Предобученный эмбеддинг оказался лучше по качеству, чем обученный нами вручную.

Далее, попробуем применить лемматизацию и удаление стоп-слов.

In [41]:
!pip install spacy

In [42]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
vocab = English()
tokenizer = vocab.tokenizer

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner',
                                            'tok2vec', 'attribute_ruler'])

In [43]:
def question_to_vec_new(question, embeddings, tokenizer, dim = 200):
    doc = nlp(question.lower())
    lemma_list = []
    for token in doc:
        if nlp.vocab[token.lemma_].is_stop == False:
          lemma_list.append(token.lemma_)

    vec = np.zeros(dim)
    num_words = 0

    for word in lemma_list:
      try:
        vec += embeddings[word]
        num_words += 1
      except:
        continue

    if num_words == 0:
      return vec

    return vec/num_words

def rank_candidates_new(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_embedding = [question_to_vec_new(question, embeddings, tokenizer, dim)]
    candidates_embeddings = [question_to_vec_new(candidate, embeddings, tokenizer, dim) for candidate in candidates]

    cos_similarity = pd.Series(cosine_similarity(question_embedding, candidates_embeddings).squeeze()).sort_values(ascending=False)
    print(cos_similarity)

    return [(i, candidates[i]) for i in cos_similarity.index]

In [44]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates_new(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


487    0.612999
619    0.599631
102    0.584694
756    0.570197
558    0.567212
         ...   
816   -0.187439
782   -0.191390
304   -0.205259
194   -0.216615
914   -0.219509
Length: 1000, dtype: float64


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


518    0.550318
495    0.544188
28     0.539057
230    0.534438
229    0.529088
         ...   
517   -0.073466
150   -0.078353
198   -0.096410
647   -0.108023
146   -0.135340
Length: 1000, dtype: float64


...

340    0.741395
220    0.645903
160    0.631969
170    0.613485
29     0.600565
         ...   
236   -0.104815
391   -0.112599
628   -0.113331
105   -0.119481
729   -0.134059
Length: 1000, dtype: float64


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


983    0.435363
653    0.381912
0      0.375215
23     0.358152
20     0.352078
         ...   
748   -0.126867
414   -0.129070
37    -0.143236
702   -0.146859
870   -0.149446
Length: 1000, dtype: float64


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


0      0.780185
601    0.554796
211    0.539580
649    0.520457
611    0.505334
         ...   
605   -0.055485
49    -0.065966
998   -0.102715
38    -0.105089
385   -0.110816
Length: 1000, dtype: float64


In [45]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.406 | Hits@   1: 0.406
DCG@   5: 0.493 | Hits@   5: 0.573
DCG@  10: 0.518 | Hits@  10: 0.648
DCG@ 100: 0.565 | Hits@ 100: 0.880
DCG@ 500: 0.578 | Hits@ 500: 0.976
DCG@1000: 0.580 | Hits@1000: 1.000


In [48]:
## попробуем только токенизацию spacy, чтобы сравнить эффективность с регулярными выражениями
def question_to_vec_new2(question, embeddings, tokenizer, dim = 200):
    doc = nlp(question.lower())
    tokens_list = []
    for token in doc:
      tokens_list.append(token.text)

    vec = np.zeros(dim)
    num_words = 0

    for word in tokens_list:
      try:
        vec += embeddings[word]
        num_words += 1
      except:
        continue

    if num_words == 0:
      return vec

    return vec/num_words

def rank_candidates_new2(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_embedding = [question_to_vec_new2(question, embeddings, tokenizer, dim)]
    candidates_embeddings = [question_to_vec_new2(candidate, embeddings, tokenizer, dim) for candidate in candidates]

    cos_similarity = pd.Series(cosine_similarity(question_embedding, candidates_embeddings).squeeze()).sort_values(ascending=False)
    print(cos_similarity)

    return [(i, candidates[i]) for i in cos_similarity.index]

In [49]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates_new2(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


487    0.611668
479    0.603320
0      0.601294
396    0.590143
102    0.586682
         ...   
668   -0.127975
816   -0.157792
194   -0.175305
304   -0.189262
914   -0.191190
Length: 1000, dtype: float64


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


380    0.576415
518    0.550318
495    0.544188
28     0.539057
230    0.534438
         ...   
234   -0.069150
517   -0.073466
198   -0.096410
647   -0.108023
146   -0.135340
Length: 1000, dtype: float64


...

340    0.713574
220    0.645903
160    0.631969
170    0.613485
29     0.600565
         ...   
236   -0.104815
391   -0.112599
628   -0.113331
105   -0.119481
729   -0.134059
Length: 1000, dtype: float64


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


983    0.398701
653    0.381912
0      0.375215
20     0.352078
491    0.350572
         ...   
748   -0.126867
414   -0.129070
37    -0.143236
870   -0.149446
702   -0.157569
Length: 1000, dtype: float64


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


0      0.763206
211    0.539580
601    0.530506
649    0.520457
611    0.505334
         ...   
529   -0.048930
605   -0.055485
49    -0.065966
38    -0.105089
385   -0.110816
Length: 1000, dtype: float64


In [50]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.406 | Hits@   1: 0.406
DCG@   5: 0.496 | Hits@   5: 0.574
DCG@  10: 0.518 | Hits@  10: 0.643
DCG@ 100: 0.563 | Hits@ 100: 0.864
DCG@ 500: 0.577 | Hits@ 500: 0.971
DCG@1000: 0.580 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Основной способ, используемый в работе - на основе регулярных выражений. На удивление, этот способ сработал лучше, чем встроенная токенизация из библиотеки spacy. Возможно, в spacy токенизации остаются знаки пунктуации, что плохо влияет на качество.

2. Нормализация слов (лемматизация), в нашем случае, повысила качество работы модели, но не сильно.

3. С задачей лучше справился скачанный эмбеддинг, а не самостоятельно обученный. Вероятно, это произошло из-за "более высокой степени обученности" скачанного эмбеддинга (большее кол-во текстов, большее кол-во нейронов на скрытом слое и т.д., что вычислительно труднее).

4. Предполагаю, что причиной плохого качества решения задачи - не лучшие, на данный момент, технологии, которые могли бы больше взять информации из обучающего датасета. + возможно повлиял небольшой размер обучающего датасета. Если бы было больше данных, то наш обученный эмбеддинг получился бы сильно лучше скачанного, т.к. обучение происходило бы на специфичных (вопросы StackOverflow) данных, что лучше бы работало на итоговых предсказаниях.

5. Вероятно, стоит использовать RNN, как говорилось в лекционном материале. Также помогла бы более глубокая предобработка данных.